# 04. Teams


## 1. Azure OpenAI 클라이언트 설정

AutoGen과 Azure OpenAI 서비스를 연동하기 위한 기본 설정입니다. 환경 변수에서 API 키와 엔드포인트 정보를 불러와 클라이언트를 초기화합니다.

In [55]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

from dotenv import load_dotenv
import os

load_dotenv()

api_version = os.getenv("AZURE_OPENAI_API_VERSION")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
azure_openai_chat_completion_client = AzureOpenAIChatCompletionClient(
            model=deployment_name,
            azure_endpoint=azure_endpoint,
            api_version=api_version,
            api_key=api_key,
        )

## 2. 단일 에이전트 팀 (Single Agent Team)

날씨 정보를 제공하는 간단한 AI 에이전트를 생성하고 실행하는 예제입니다. 
- `get_weather` 함수: 도시명을 입력받아 날씨 정보를 반환하는 도구
- `AssistantAgent`: 한국어로 대화하며 날씨 정보를 제공하는 에이전트
- `RoundRobinGroupChat`: 단일 에이전트로 구성된 팀 생성

In [56]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

# Create an OpenAI model client.
model_client = azure_openai_chat_completion_client

# Define a tool that gets the weather for a city.
async def get_weather(city: str) -> str:
    """Get the weather for a city."""
    return f"{city}의 날씨는 섭씨 22도이고 맑습니다."


# Create an assistant agent.
weather_agent = AssistantAgent(
    "assistant",
    model_client=model_client,
    tools=[get_weather],
    system_message="작업이 완료되면 '완료'라고 응답하세요. 정중하고 친근한 한국어로 대화하세요.",
)

# Define a termination condition.
text_termination = TextMentionTermination("완료")

# Create a single-agent team.
single_agent_team = RoundRobinGroupChat([weather_agent], termination_condition=text_termination)

async def run_team() -> None:
    result = await single_agent_team.run(task="서울의 날씨는 어떤가요?")
    print(result)


# Use `asyncio.run(run_team())` when running in a script.
await run_team()

messages=[TextMessage(id='38448310-dec1-4cb0-b162-c37cfc8cf16c', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 15, 36, 18, 415076, tzinfo=datetime.timezone.utc), content='서울의 날씨는 어떤가요?', type='TextMessage'), ToolCallRequestEvent(id='4a0b4172-d0ab-4384-9717-e5829d7e55b9', source='assistant', models_usage=RequestUsage(prompt_tokens=88, completion_tokens=15), metadata={}, created_at=datetime.datetime(2025, 11, 1, 15, 36, 19, 70767, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_eK0QgGSByk3czi5tYiKC14xX', arguments='{"city":"서울"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='e3ddb6b1-3392-471f-b389-b89ec1f3784d', source='assistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 15, 36, 19, 72711, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='서울의 날씨는 섭씨 22도이고 맑습니다.', name='get_weather', call_id='call_eK0QgGSByk3czi5tYiKC14xX', is_error=False)], 

## 3. 스트리밍 모드로 팀 실행

동일한 단일 에이전트 팀을 스트리밍 모드로 실행하는 방법입니다. 실시간으로 메시지를 받아보고 작업 완료 상태를 확인할 수 있습니다.
- `run_stream()`: 실시간 메시지 스트리밍
- `TaskResult`: 작업 완료 시 중단 이유 출력

In [57]:
from autogen_agentchat.base import TaskResult


async def run_team_stream() -> None:
    async for message in single_agent_team.run_stream(task="부산의 날씨는 어떤가요?"):
        if isinstance(message, TaskResult):
            print("중단 이유:", message.stop_reason)
        else:
            print(message)


# Use `asyncio.run(run_team_stream())` when running in a script.
await run_team_stream()

id='a688b27a-52bb-463a-a344-7d85f39e013c' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 11, 1, 15, 36, 20, 175281, tzinfo=datetime.timezone.utc) content='부산의 날씨는 어떤가요?' type='TextMessage'
id='ec716739-2772-4b2c-8644-572730cd9070' source='assistant' models_usage=RequestUsage(prompt_tokens=174, completion_tokens=16) metadata={} created_at=datetime.datetime(2025, 11, 1, 15, 36, 20, 725362, tzinfo=datetime.timezone.utc) content=[FunctionCall(id='call_iEOxvCo8c3gc4yzF793FPVYa', arguments='{"city":"부산"}', name='get_weather')] type='ToolCallRequestEvent'
id='19d5fc71-755c-4a72-beba-9dc91675fa70' source='assistant' models_usage=None metadata={} created_at=datetime.datetime(2025, 11, 1, 15, 36, 20, 726844, tzinfo=datetime.timezone.utc) content=[FunctionExecutionResult(content='부산의 날씨는 섭씨 22도이고 맑습니다.', name='get_weather', call_id='call_iEOxvCo8c3gc4yzF793FPVYa', is_error=False)] type='ToolCallExecutionEvent'
id='e853aa26-1e82-449f-b4c7-a7004cbdae5b' source='assis

## 4. 콘솔 UI를 활용한 대화형 실행

AutoGen의 콘솔 UI를 사용하여 더 직관적으로 에이전트와의 대화를 확인할 수 있습니다.
- `Console()`: 대화형 콘솔 인터페이스 제공
- `reset()`: 팀 상태 초기화로 새로운 대화 시작
- 영어 예제로 다국어 지원 확인

In [58]:
from autogen_agentchat.ui import Console

# Use `asyncio.run(single_agent_team.reset())` when running in a script.
await single_agent_team.reset()  # Reset the team for the next run.
# Use `asyncio.run(single_agent_team.run_stream(task="What is the weather in Seattle?"))` when running in a script.
await Console(
    single_agent_team.run_stream(task="What is the weather in Seattle?")
)  # Stream the messages to the console.

---------- TextMessage (user) ----------
What is the weather in Seattle?


---------- ToolCallRequestEvent (assistant) ----------
[FunctionCall(id='call_xjL8IA7tf7TVRvvLSzJaYbGK', arguments='{"city":"Seattle"}', name='get_weather')]
---------- ToolCallExecutionEvent (assistant) ----------
[FunctionExecutionResult(content='Seattle의 날씨는 섭씨 22도이고 맑습니다.', name='get_weather', call_id='call_xjL8IA7tf7TVRvvLSzJaYbGK', is_error=False)]
---------- ToolCallSummaryMessage (assistant) ----------
Seattle의 날씨는 섭씨 22도이고 맑습니다.
---------- TextMessage (assistant) ----------
시애틀의 날씨는 현재 맑고 기온은 섭씨 22도입니다. 다른 도움이 필요하시면 말씀해 주세요!
---------- TextMessage (assistant) ----------
완료


TaskResult(messages=[TextMessage(id='361a5fd3-91b7-4719-a232-aa8bb2a6d720', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 15, 36, 21, 928531, tzinfo=datetime.timezone.utc), content='What is the weather in Seattle?', type='TextMessage'), ToolCallRequestEvent(id='96f05393-13f2-4520-8e2b-d956e6e57ea5', source='assistant', models_usage=RequestUsage(prompt_tokens=86, completion_tokens=15), metadata={}, created_at=datetime.datetime(2025, 11, 1, 15, 36, 22, 439760, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_xjL8IA7tf7TVRvvLSzJaYbGK', arguments='{"city":"Seattle"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='08359d03-02b3-428d-8e79-7b17857a93f6', source='assistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 15, 36, 22, 440992, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='Seattle의 날씨는 섭씨 22도이고 맑습니다.', name='get_weather', call_id='call_xjL8

## 5. Reflection Team - 다중 에이전트 협업

두 개의 에이전트가 협업하여 작업을 수행하는 반성 패턴입니다.
- **Primary Agent**: 주요 작업을 수행하는 AI 어시스턴트
- **Critic Agent**: 결과물을 검토하고 건설적인 피드백 제공
- **협업 프로세스**: Primary가 작업 → Critic이 검토 → 피드백 반영 → 승인 시 완료
- **종료 조건**: "승인합니다" 키워드 또는 최대 15개 메시지 제한

In [59]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an OpenAI model client.
model_client = azure_openai_chat_completion_client
# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="당신은 도움이 되는 AI 어시스턴트입니다. 정중하고 친근한 한국어로 대화하며, 정확하고 유용한 정보를 제공하세요.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="건설적인 피드백을 제공하세요. 피드백이 반영되었을 때 '승인합니다'라고 응답하세요. 정중하고 예의바른 한국어로 소통하세요.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("승인합니다")
# Define a termination condition that stops the task after 5 messages.
max_message_termination = MaxMessageTermination(15)
# Combine the termination conditions using the `|`` operator so that the
# task stops when either condition is met.
termination = text_termination | max_message_termination

# Create a team with the primary and critic agents.
reflection_team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=termination)

### 5-1. Reflection Team 실행 - 가을 시 창작

Reflection Team을 실행하여 가을에 대한 시를 창작하는 예제입니다. Primary 에이전트가 시를 작성하면 Critic 에이전트가 검토하고 피드백을 제공합니다.

In [60]:

# Use `asyncio.run(Console(reflection_team.run_stream(task="Write a short poem about fall season.")))` when running in a script.
await Console(
    reflection_team.run_stream(task="가을에 대한 짧은 시를 써보세요.")
)  # Stream the messages to the console.

---------- TextMessage (user) ----------
가을에 대한 짧은 시를 써보세요.


---------- TextMessage (primary) ----------
가을의 바람 속에  
붉은 단풍이 흩날리고,  
가벼운 발걸음 따라  
추억이 그려지네.  

차가운 공기 속에서  
따뜻한 커피 한 잔,  
소중한 사람과 함께  
가을의 정취를 나누네.  
---------- TextMessage (critic) ----------
아주 아름다운 시입니다! 가을의 풍경과 감정이 잘 표현되어 있어요. 단풍과 커피의 이미지가 따뜻함을 주네요. 다만, 마지막 구절에서 '가을의 정취를 나누네' 부분을 좀 더 구체적이거나 독특한 방식으로 표현해보면 좋을 것 같아요. 예를 들어, '가을의 이야기를 나누네'처럼요. 이런 수정이 이루어진다면 더욱 깊이 있는 감상이 될 것 같습니다. 더 많은 시를 기대합니다!
---------- TextMessage (primary) ----------
칭찬해 주셔서 감사합니다! 귀하의 피드백을 반영하여 시를 수정해 보겠습니다.

---

가을의 바람 속에  
붉은 단풍이 흩날리고,  
가벼운 발걸음 따라  
추억이 그려지네.  

차가운 공기 속에서  
따뜻한 커피 한 잔,  
소중한 사람과 함께  
가을의 이야기를 나누네.  

---

더 많은 시나 다른 주제에 대한 요청이 있으시면 언제든지 말씀해 주세요!
---------- TextMessage (critic) ----------
수고하셨습니다! 수정하신 시가 훨씬 더 풍부하고 감정적으로 다가오네요. '가을의 이야기를 나누네'라는 표현이 특히 마음에 듭니다. 가을의 정취가 잘 살아나고, 읽는 이에게 따뜻한 느낌을 전달합니다. 정말 잘하셨어요! 

더 많은 시나 다른 주제에 대한 요청이 있으면 언제든지 말씀해 주세요. 계속해서 창작해 주시는 걸 기대하겠습니다! 승인을 합니다.
---------- TextMessage (primary) ----------
너무 고맙습니다! 따뜻한 말씀에 힘이 납니다. 가을의 감성을 더 깊게 전달할 수 있어서 기쁩니다. 언제든지 새로운 시나 

TaskResult(messages=[TextMessage(id='1e171cb1-41ad-4dd7-946a-364fc11af2bd', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 15, 36, 23, 676037, tzinfo=datetime.timezone.utc), content='가을에 대한 짧은 시를 써보세요.', type='TextMessage'), TextMessage(id='97007eec-3646-48af-8d40-53de628f0273', source='primary', models_usage=RequestUsage(prompt_tokens=61, completion_tokens=75), metadata={}, created_at=datetime.datetime(2025, 11, 1, 15, 36, 24, 829955, tzinfo=datetime.timezone.utc), content='가을의 바람 속에  \n붉은 단풍이 흩날리고,  \n가벼운 발걸음 따라  \n추억이 그려지네.  \n\n차가운 공기 속에서  \n따뜻한 커피 한 잔,  \n소중한 사람과 함께  \n가을의 정취를 나누네.  ', type='TextMessage'), TextMessage(id='899cce6c-8413-47c2-b70c-f668168fe838', source='critic', models_usage=RequestUsage(prompt_tokens=150, completion_tokens=120), metadata={}, created_at=datetime.datetime(2025, 11, 1, 15, 36, 26, 349356, tzinfo=datetime.timezone.utc), content="아주 아름다운 시입니다! 가을의 풍경과 감정이 잘 표현되어 있어요. 단풍과 커피의 이미지가 따뜻함을 주네요. 다만, 마지막 구절에서 '가을의 정취를 

### 5-2. Reflection Team 실행 - 당나라 시풍 창작

더 복잡한 창작 작업으로 중국 당나라 시풍의 시를 창작하는 예제입니다. 특정 문학적 스타일을 요구하는 고급 작업에서 두 에이전트가 어떻게 협업하는지 확인할 수 있습니다.

In [ ]:
await Console(reflection_team.run_stream(task="중국 당나라 시풍으로 시를 써보세요."))

---------- TextMessage (user) ----------
중국 당나라 시풍으로 시를 써보세요.
---------- TextMessage (primary) ----------
중국 당나라 시풍을 참고하여 한 편의 시를 써보겠습니다. 당나라 시는 자연을 찬미하고 인간의 감정을 섬세하게 표현하는 특징이 있습니다. 

---

가을 하늘 푸르르고,  
구름은 가벼워라.  
단풍은 분홍빛 열매처럼,  
잎사귀에 숨어 있는 꿈.  

산의 길은 흐트러져,  
이야기하는 듯 한가로이.  
시냇물은 눈을 감고,  
여유로이 흐르는 그 모습.  

별빛 아래 고요한 밤,  
하늘과 땅이 아는 기억.  
내 마음의 소식이  
종소리처럼 울리네.  

---

이 시가 당나라 시풍의 느낌을 잘 담아냈기를 바랍니다. 다른 주제나 스타일에 대한 요청이 있다면 언제든지 말씀해주세요!
---------- TextMessage (critic) ----------
아주 멋진 시입니다! 당나라 시풍의 특징이 잘 드러나 있어 감탄을 자아냅니다. 특히 자연을 찬미하는 모습과 섬세한 감정이 조화롭게 어우러져 있습니다.

첫 번째 구절에서 '가을 하늘 푸르르고, 구름은 가벼워라'라는 표현이 생생하게 이미지가 떠오르네요. 또한, '이야기하는 듯 한가로이'라는 표현이 산의 평화로운 분위기를 잘 담고 있습니다. 그리고 '내 마음의 소식이 종소리처럼 울리네'라는 마무리로 마음의 깊은 감정을 느끼게 만들어주네요.

당나라 시풍의 특징인 자연과 인간의 감정, 그리고 그 여유로운 느낌을 잘 살려주셔서 정말 훌륭합니다! 다른 주제에 대한 요청이나 추가적인 수정이 필요하시다면 언제든 말씀해 주세요. 다음 시가 더욱 기대됩니다!
---------- TextMessage (primary) ----------
따뜻한 칭찬에 감사드립니다! 당나라 시풍의 정서와 자연의 아름다움을 잘 표현할 수 있어서 기쁩니다. 여러분의 피드백이 저에게 큰 힘이 되어 창작에 더욱 열정을 쏟게 됩니다.

더 다양한 주제나 스타

## 6. 핸드오프 팀 (Handoff Team) - 사용자 전달 패턴

답변을 모르는 질문이나 복잡한 요청을 사용자에게 전달하는 패턴입니다.
- **Lazy Agent**: 답변을 모를 때 사용자에게 전달하는 에이전트
- **Handoff 메커니즘**: 특정 조건에서 사용자에게 제어권 전달
- **실용적 활용**: 에이전트의 한계를 인정하고 인간의 도움을 요청하는 겸손한 AI 패턴

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import Handoff
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

# Create an OpenAI model client.
model_client = azure_openai_chat_completion_client

# Create a lazy assistant agent that always hands off to the user.
lazy_agent = AssistantAgent(
    "lazy_assistant",
    model_client=model_client,
    handoffs=[Handoff(target="user", message="사용자에게 전달합니다.")],
    system_message="답을 모를 때는 항상 사용자에게 전달하세요. 작업이 완료되면 '완료'라고 응답하세요. 정중하고 친근한 한국어로 대화하세요.",
)

# Define a termination condition that checks for handoff message targetting helper and text "TERMINATE".
handoff_termination = HandoffTermination(target="user")
text_termination = TextMentionTermination("완료")
termination = handoff_termination | text_termination

# Create a single-agent team.
lazy_agent_team = RoundRobinGroupChat([lazy_agent], termination_condition=termination)

### 6-1. 핸드오프 팀 실행 예제

두 가지 다른 유형의 질문으로 핸드오프 팀의 동작을 테스트합니다:
1. **"서울 날씨 어때?"** - 구체적인 날씨 질문 (통계 정보 포함)
2. **"날씨 맑은데?"** - 모호한 질문으로 사용자 전달 가능성 테스트

각 질문에 대해 에이전트가 어떻게 반응하고 언제 사용자에게 전달하는지 관찰할 수 있습니다.

In [ ]:
from autogen_agentchat.ui import Console

await Console(lazy_agent_team.run_stream(task="서울 날씨 어때?"),  output_stats=True)

# Handoff to the user before answering the question

await Console(lazy_agent_team.run_stream(task="날씨 맑은데?"))

---------- TextMessage (user) ----------
서울 날씨 어때?
---------- ToolCallRequestEvent (lazy_assistant) ----------
[FunctionCall(id='call_xaXRYh6f2pZZwj5JYxlo6VS2', arguments='{}', name='transfer_to_user')]
[Prompt tokens: 83, Completion tokens: 12]
---------- ToolCallExecutionEvent (lazy_assistant) ----------
[FunctionExecutionResult(content='사용자에게 전달합니다.', name='transfer_to_user', call_id='call_xaXRYh6f2pZZwj5JYxlo6VS2', is_error=False)]
---------- HandoffMessage (lazy_assistant) ----------
사용자에게 전달합니다.
---------- Summary ----------
Number of messages: 4
Finish reason: Handoff to user from lazy_assistant detected.
Total prompt tokens: 83
Total completion tokens: 12
Duration: 1.38 seconds
---------- TextMessage (user) ----------
날씨 맑은데?
---------- TextMessage (lazy_assistant) ----------
네, 서울의 날씨가 맑다면 기분이 좋을 것 같네요! 외출하기에도 좋은 날씨군요. 특별한 계획이 있으신가요?
---------- TextMessage (lazy_assistant) ----------
완료


TaskResult(messages=[TextMessage(id='ab4df81e-c621-45a2-9e72-4d29a726e275', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 14, 23, 1, 160258, tzinfo=datetime.timezone.utc), content='날씨 맑은데?', type='TextMessage'), TextMessage(id='75afbf55-1c3d-4f3b-a424-4e47c42443a8', source='lazy_assistant', models_usage=RequestUsage(prompt_tokens=121, completion_tokens=40), metadata={}, created_at=datetime.datetime(2025, 11, 1, 14, 23, 2, 152023, tzinfo=datetime.timezone.utc), content='네, 서울의 날씨가 맑다면 기분이 좋을 것 같네요! 외출하기에도 좋은 날씨군요. 특별한 계획이 있으신가요?', type='TextMessage'), TextMessage(id='6b07f2e4-5401-446d-8a6e-0dfbe9248b73', source='lazy_assistant', models_usage=RequestUsage(prompt_tokens=163, completion_tokens=4), metadata={}, created_at=datetime.datetime(2025, 11, 1, 14, 23, 2, 681238, tzinfo=datetime.timezone.utc), content='완료', type='TextMessage')], stop_reason="Text '완료' mentioned")

---

# 종합 실습 과제: 실시간 API 연동 스마트 여행 계획 AI 팀 만들기

지금까지 학습한 내용을 바탕으로 **실제 API를 활용한** 여행 계획 다중 에이전트 시스템을 만들어보겠습니다.

## 📋 실습 목표 & 실제 API 연동
1. **여행지 전문가**: 🌐 **Google Places API**로 실시간 명소/맛집/숙박 추천
2. **날씨 컨설턴트**: 🌤️ **OpenWeatherMap API**로 실시간 날씨 예보 및 여행 조언
3. **예산 관리자**: 💰 여행 비용 계산 및 예산 조언  
4. **여행 컨시어지**: 🛎️ 복잡한 요청 시 전문가에게 핸드오프

## 🆕 API 활용 특징
- **Google Places API**: 실제 관광지, 맛집, 숙박시설 정보 + 평점
- **OpenWeatherMap API**: 실시간 날씨 예보 + 날씨 기반 여행 추천
- **Fallback 시스템**: API 키 없이도 기본 데이터로 학습 가능

이 실습을 통해 **실제 API 연동**, **단일 에이전트**, **반성 팀**, **핸드오프 패턴**을 모두 활용하는 종합적인 시스템을 구축해보세요!

In [ ]:
# ============================================================================
# 🎯 실습 과제: 스마트 여행 계획 AI 팀 구현하기
# ============================================================================

# 필요한 라이브러리 import
import aiohttp
import json
import random
from datetime import datetime, timedelta

# 1. 여행지 추천 도구 함수들 정의 (Google Places API 활용)
async def get_popular_attractions(city: str, category: str = "관광지") -> str:
    """Google Places API를 사용해서 실제 명소를 검색합니다."""
    google_api_key = os.getenv("GOOGLE_PLACES_API_KEY")
    
    if not google_api_key:
        # API 키가 없으면 기본 데이터베이스 사용
        attractions_db = {
            "서울": {
                "관광지": ["경복궁", "남산타워", "명동", "홍대", "동대문디자인플라자"],
                "맛집": ["광장시장", "명동교자", "신촌맛집거리", "이태원"],
                "숙박": ["롯데호텔", "신라호텔", "게스트하우스", "에어비앤비"]
            },
            "부산": {
                "관광지": ["해운대해수욕장", "광안대교", "감천문화마을", "태종대", "광안리해수욕장"],
                "맛집": ["자갈치시장", "해운대맛집거리", "서면맛집"],
                "숙박": ["해운대호텔", "부산역근처숙소", "게스트하우스"]
            },
            "제주": {
                "관광지": ["성산일출봉", "한라산", "우도", "천지연폭포", "협재해수욕장"],
                "맛집": ["흑돼지거리", "동문시장", "제주향토음식"],
                "숙박": ["제주신화월드", "펜션", "리조트"]
            },
            "경주": {
                "관광지": ["불국사", "석굴암", "첨성대", "안압지", "대릉원"],
                "맛집": ["경주전통음식", "황남빵", "보리밥집"],
                "숙박": ["경주호텔", "한옥펜션", "게스트하우스"]
            }
        }
        
        city_data = attractions_db.get(city, {"관광지": ["정보를 찾을 수 없습니다"]})
        results = city_data.get(category, ["해당 카테고리 정보가 없습니다"])
        
        return f"{city}의 {category} 추천:\n" + "\n".join([f"- {item}" for item in results])
    
    try:
        # Google Places API를 사용한 실제 검색
        place_type_mapping = {
            "관광지": "tourist_attraction",
            "맛집": "restaurant", 
            "숙박": "lodging",
            "쇼핑": "shopping_mall",
            "카페": "cafe"
        }
        
        place_type = place_type_mapping.get(category, "tourist_attraction")
        
        async with aiohttp.ClientSession() as session:
            # Text Search API 사용
            search_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
            params = {
                "query": f"{city} {category}",
                "type": place_type,
                "key": google_api_key,
                "language": "ko"
            }
            
            async with session.get(search_url, params=params) as response:
                if response.status != 200:
                    return f"죄송합니다. {city}의 {category} 정보를 가져올 수 없습니다."
                
                data = await response.json()
                
                if not data.get("results"):
                    return f"{city}에서 {category} 정보를 찾을 수 없습니다."
                
                # 상위 10개 결과 추출
                places = []
                for place in data["results"][:10]:
                    name = place.get("name", "이름 없음")
                    rating = place.get("rating", 0)
                    
                    place_info = f"{name}"
                    if rating > 0:
                        place_info += f" (평점: {rating})"
                    places.append(place_info)
                
                result = f"{city}의 {category} 추천 (Google Places API):\n"
                result += "\n".join([f"- {place}" for place in places])
                
                return result
                
    except Exception as e:
        return f"{category} 정보를 가져오는 중 오류가 발생했습니다: {str(e)}"

async def calculate_travel_budget(city: str, days: int, style: str = "보통") -> str:
    """여행 예산을 계산합니다."""
    base_costs = {
        "서울": {"숙박": 80000, "식사": 50000, "교통": 30000, "관광": 40000},
        "부산": {"숙박": 60000, "식사": 40000, "교통": 25000, "관광": 35000},
        "제주": {"숙박": 100000, "식사": 60000, "교통": 40000, "관광": 50000},
        "경주": {"숙박": 50000, "식사": 35000, "교통": 20000, "관광": 30000},
    }
    
    if city not in base_costs:
        return f"{city}의 예산 정보를 찾을 수 없습니다."
    
    multiplier = {"절약": 0.7, "보통": 1.0, "럭셔리": 1.5}.get(style, 1.0)
    daily_cost = sum(base_costs[city].values()) * multiplier
    total_cost = daily_cost * days
    
    return f"{city} {days}일 여행 ({style} 스타일): 총 {total_cost:,.0f}원 (일일 {daily_cost:,.0f}원)"

async def get_weather_recommendation(city: str, days: int = 3) -> str:
    """OpenWeatherMap API를 사용해서 날씨 예보와 여행 추천사항을 제공합니다."""
    api_key = os.getenv("OPENWEATHER_API_KEY")
    
    if not api_key:
        # API 키가 없으면 모의 데이터 제공
        weather_conditions = ["맑음", "흐림", "비", "눈", "안개"]
        temperatures = list(range(15, 30))
        
        forecast = {"city": city, "forecast": []}
        
        for i in range(days):
            date = datetime.now() + timedelta(days=i)
            day_weather = {
                "date": date.strftime("%Y-%m-%d"),
                "condition": random.choice(weather_conditions),
                "temperature": f"{random.choice(temperatures)}°C",
                "humidity": f"{random.randint(40, 80)}%"
            }
            forecast["forecast"].append(day_weather)
        
        # 계절별 추천사항 추가
        current_month = datetime.now().month
        if current_month in [3, 4, 5]:
            season = "봄"
        elif current_month in [6, 7, 8]:
            season = "여름"
        elif current_month in [9, 10, 11]:
            season = "가을"
        else:
            season = "겨울"
            
        recommendations = {
            ("서울", "봄"): "벚꽃이 아름다운 계절입니다. 여의도 한강공원 추천!",
            ("서울", "여름"): "더위를 피해 실내 관광지나 한강 야경을 즐기세요.",
            ("서울", "가을"): "단풍이 아름다운 남산과 북촌한옥마을을 추천합니다.",
            ("서울", "겨울"): "스케이트장과 크리스마스 마켓을 즐기세요.",
            ("부산", "여름"): "해수욕장과 바다 활동을 즐기기 최적의 시기입니다!",
            ("제주", "봄"): "유채꽃과 벚꽃이 아름다운 계절입니다.",
        }
        
        season_tip = recommendations.get((city, season), f"{city}의 {season} 여행을 즐기세요!")
        
        result = json.dumps(forecast, ensure_ascii=False, indent=2)
        result += f"\n\n🌟 계절별 추천: {season_tip}"
        
        return result
    
    try:
        # 실제 OpenWeatherMap API 호출
        async with aiohttp.ClientSession() as session:
            # 1. 도시 좌표 얻기
            geo_url = f"http://api.openweathermap.org/geo/1.0/direct?q={city}&limit=1&appid={api_key}"
            async with session.get(geo_url) as response:
                if response.status != 200:
                    return f"죄송합니다. {city}의 위치 정보를 찾을 수 없습니다."
                
                geo_data = await response.json()
                if not geo_data:
                    return f"죄송합니다. {city}의 위치 정보를 찾을 수 없습니다."
                
                lat, lon = geo_data[0]['lat'], geo_data[0]['lon']
            
            # 2. 날씨 예보 가져오기
            weather_url = f"http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={api_key}&units=metric&lang=kr"
            async with session.get(weather_url) as response:
                if response.status != 200:
                    return f"죄송합니다. {city}의 날씨 정보를 가져올 수 없습니다."
                
                weather_data = await response.json()
                
                # 일별 예보 정리
                daily_forecasts = {}
                for item in weather_data['list'][:days*8]:  # 3시간 간격이므로 하루 8개
                    date = item['dt_txt'][:10]  # YYYY-MM-DD
                    if date not in daily_forecasts:
                        daily_forecasts[date] = {
                            "date": date,
                            "condition": item['weather'][0]['description'],
                            "temperature": f"{int(item['main']['temp'])}°C",
                            "humidity": f"{item['main']['humidity']}%"
                        }
                
                forecast = {
                    "city": city,
                    "forecast": list(daily_forecasts.values())[:days]
                }
                
                # 날씨 기반 여행 추천 추가
                avg_temp = sum([int(d['temperature'][:-2]) for d in forecast['forecast']]) / len(forecast['forecast'])
                
                if avg_temp > 25:
                    weather_tip = "더운 날씨예요! 시원한 실내 관광지나 수영장, 해변 활동을 추천합니다."
                elif avg_temp < 10:
                    weather_tip = "추운 날씨예요! 따뜻한 실내 명소나 온천, 찜질방을 추천합니다."
                else:
                    weather_tip = "야외 활동하기 좋은 날씨입니다! 산책로나 공원 관광을 즐기세요."
                
                result = json.dumps(forecast, ensure_ascii=False, indent=2)
                result += f"\n\n🌟 날씨 기반 추천: {weather_tip}"
                
                return result
                
    except Exception as e:
        return f"날씨 정보를 가져오는 중 오류가 발생했습니다: {str(e)}"

In [ ]:
# 2. 여행지 전문가 에이전트 (단일 에이전트 팀)
travel_expert = AssistantAgent(
    "travel_expert",
    model_client=azure_openai_chat_completion_client,
    tools=[get_popular_attractions, get_weather_recommendation],
    system_message="""당신은 한국 여행 전문가입니다. 
    - Google Places API와 OpenWeatherMap API를 활용해 실시간 정보를 제공하세요
    - 고객의 여행 목적지에 대한 실제 관광명소, 맛집, 숙박시설을 추천하세요
    - 현재 날씨 상황을 고려한 여행 팁과 주의사항을 제공하세요
    - get_popular_attractions 함수는 city와 category(관광지, 맛집, 숙박 등) 파라미터를 받습니다
    - get_weather_recommendation 함수는 city와 days 파라미터를 받아 날씨 예보와 추천사항을 제공합니다
    - 정중하고 친근한 한국어로 대화하며, 작업 완료 시 '추천 완료'라고 말하세요""",
)

# 여행지 전문가 팀 생성
expert_termination = TextMentionTermination("추천 완료")
travel_expert_team = RoundRobinGroupChat([travel_expert], termination_condition=expert_termination)

In [ ]:
# 3. 예산 관리 팀 (반성 패턴 - Primary + Critic)

# 예산 계산 에이전트
budget_planner = AssistantAgent(
    "budget_planner", 
    model_client=azure_openai_chat_completion_client,
    tools=[calculate_travel_budget],
    system_message="""당신은 여행 예산 전문가입니다.
    - 고객의 여행 계획에 맞는 현실적인 예산을 계산하세요
    - 절약 팁과 예산 최적화 방안을 제안하세요
    - 정확한 숫자와 근거를 제시하며 정중한 한국어로 소통하세요""",
)

# 예산 검토 에이전트  
budget_critic = AssistantAgent(
    "budget_critic",
    model_client=azure_openai_chat_completion_client,
    system_message="""당신은 여행 예산 검토 전문가입니다.
    - 제시된 예산이 현실적이고 합리적인지 검토하세요
    - 누락된 비용이나 과도한 예산이 있는지 점검하세요
    - 검토 완료 시 '예산 승인합니다'라고 응답하세요
    - 건설적인 피드백을 정중한 한국어로 제공하세요""",
)

# 예산 관리 팀 생성 (반성 패턴)
budget_termination = TextMentionTermination("예산 승인합니다") | MaxMessageTermination(10)
budget_management_team = RoundRobinGroupChat([budget_planner, budget_critic], termination_condition=budget_termination)

In [ ]:
# 4. 여행 컨시어지 (핸드오프 패턴)

travel_concierge = AssistantAgent(
    "travel_concierge",
    model_client=azure_openai_chat_completion_client,
    handoffs=[Handoff(target="travel_advisor", message="전문 상담사에게 연결합니다")],
    system_message="""당신은 여행 컨시어지입니다.
    - 간단한 질문은 직접 답변하고, 복잡하거나 전문적인 상담이 필요한 경우 전문가에게 연결하세요
    - 항공편, 비자, 보험 등 전문 지식이 필요한 문의는 전문가에게 전달하세요  
    - 작업 완료 시 '안내 완료'라고 응답하세요
    - 항상 정중하고 도움이 되는 한국어로 소통하세요""",
)

# 컨시어지 팀 생성
concierge_termination = HandoffTermination(target="travel_advisor") | TextMentionTermination("안내 완료")
concierge_team = RoundRobinGroupChat([travel_concierge], termination_condition=concierge_termination)

## 🧪 실습 테스트: 실시간 API 연동 기능 확인

이제 만든 **실시간 API 연동 AI 팀들**을 실제로 테스트해보겠습니다. 각 팀이 어떻게 실제 데이터로 작동하는지 관찰해보세요!

### 📡 API 연동 상태
- **Google Places API**: 실제 관광지/맛집 검색 (API 키 있을 경우)
- **OpenWeatherMap API**: 실시간 날씨 예보 (API 키 있을 경우)  
- **Fallback**: API 키가 없어도 기본 데이터로 동작

In [ ]:
# 테스트 1: 여행지 전문가 팀 - Google Places API + OpenWeatherMap API 활용
print("=== 🗺️ 여행지 전문가 팀 테스트 (실시간 API 연동) ===")
await Console(travel_expert_team.run_stream(task="부산 여행 계획 중인데, 실제 관광명소와 맛집 추천하고 현재 날씨도 확인해주세요!"))

=== 🗺️ 여행지 전문가 팀 테스트 (실시간 API 연동) ===
---------- TextMessage (user) ----------
부산 여행 계획 중인데, 실제 관광명소와 맛집 추천하고 현재 날씨도 확인해주세요!


---------- ToolCallRequestEvent (travel_expert) ----------
[FunctionCall(id='call_ppeJlXCpoq7rsAiM9f6fq2OM', arguments='{"city": "부산", "category": "관광지"}', name='get_popular_attractions'), FunctionCall(id='call_VXtiaeyEQWOQ7t1vharTk2bE', arguments='{"city": "부산", "category": "맛집"}', name='get_popular_attractions'), FunctionCall(id='call_w0zEW3HC8bywk9wI8Jxs9zjI', arguments='{"city": "부산"}', name='get_weather_recommendation')]
---------- ToolCallExecutionEvent (travel_expert) ----------
[FunctionExecutionResult(content='부산의 관광지 추천 (Google Places API):\n- 감천문화마을 (평점: 4.4)\n- 용두산공원 (평점: 4.3)\n- BIFF광장 (평점: 4.1)\n- 오륙도 스카이워크 (평점: 4.2)\n- 청사포 (평점: 4.4)\n- 해동용궁사 (평점: 4.4)\n- 송도용궁구름다리 (평점: 4.3)\n- 흰여울문화마을 (평점: 4.4)\n- 40계단 문화관광테마거리 (평점: 3.9)\n- 부산시민공원 (평점: 4.6)', name='get_popular_attractions', call_id='call_ppeJlXCpoq7rsAiM9f6fq2OM', is_error=False), FunctionExecutionResult(content='부산의 맛집 추천 (Google Places API):\n- 연화덮밥 (평점: 4.9)\n- 동녘돈까스 송도본점 (평점: 4.9)\n- 술고당 (평점: 4.8)\n- 신촌양푼이갈비찜 서면본점 (평점

TaskResult(messages=[TextMessage(id='7a53fa30-1a4f-467c-80d0-62f18d4c4407', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 14, 21, 38, 614284, tzinfo=datetime.timezone.utc), content='부산 여행 계획 중인데, 실제 관광명소와 맛집 추천하고 현재 날씨도 확인해주세요!', type='TextMessage'), ToolCallRequestEvent(id='d844a75a-8b08-4c09-96f4-611b783e70ee', source='travel_expert', models_usage=RequestUsage(prompt_tokens=314, completion_tokens=82), metadata={}, created_at=datetime.datetime(2025, 11, 1, 14, 21, 40, 221299, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_ppeJlXCpoq7rsAiM9f6fq2OM', arguments='{"city": "부산", "category": "관광지"}', name='get_popular_attractions'), FunctionCall(id='call_VXtiaeyEQWOQ7t1vharTk2bE', arguments='{"city": "부산", "category": "맛집"}', name='get_popular_attractions'), FunctionCall(id='call_w0zEW3HC8bywk9wI8Jxs9zjI', arguments='{"city": "부산"}', name='get_weather_recommendation')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='1224fe6

In [ ]:
# 테스트 2: 예산 관리 팀 - 반성 패턴 (협업)
print("\n=== 💰 예산 관리 팀 테스트 (Primary + Critic 협업) ===")
await Console(budget_management_team.run_stream(task="서울 3박 4일 가족 여행 예산을 계산해주세요. 럭셔리 스타일로 가고 싶어요!"))


=== 💰 예산 관리 팀 테스트 (Primary + Critic 협업) ===
---------- TextMessage (user) ----------
서울 3박 4일 가족 여행 예산을 계산해주세요. 럭셔리 스타일로 가고 싶어요!
---------- ToolCallRequestEvent (budget_planner) ----------
[FunctionCall(id='call_S4x5rGHYGXGHsRVOHgTtmi3o', arguments='{"city":"서울","days":4,"style":"럭셔리"}', name='calculate_travel_budget')]
---------- ToolCallExecutionEvent (budget_planner) ----------
[FunctionExecutionResult(content='서울 4일 여행 (럭셔리 스타일): 총 1,200,000원 (일일 300,000원)', name='calculate_travel_budget', call_id='call_S4x5rGHYGXGHsRVOHgTtmi3o', is_error=False)]
---------- ToolCallSummaryMessage (budget_planner) ----------
서울 4일 여행 (럭셔리 스타일): 총 1,200,000원 (일일 300,000원)
---------- TextMessage (budget_critic) ----------
제시된 예산은 하루에 300,000원을 포함해 총 1,200,000원으로 계산되었습니다. 그러나 럭셔리 스타일의 여행을 고려할 때, 일부 항목에 대해 더 상세히 검토해 보아야 할 것 같습니다. 다음은 몇 가지 고려해야 할 사항입니다:

1. **숙박:** 럭셔리 스타일일 경우, 고급 호텔의 숙박비가 포함되어야 합니다. 일반적으로 1박당 20만 원에서 50만 원 이상이 될 수 있습니다. 3박 기준으로 최소 60만 원에서 150만 원 정도의 예산이 필요합니다.

2. **식사:** 고급 레스토랑에서의 식사

TaskResult(messages=[TextMessage(id='ee36d8a1-0c56-46c7-b877-4c9a79860092', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 14, 21, 48, 397306, tzinfo=datetime.timezone.utc), content='서울 3박 4일 가족 여행 예산을 계산해주세요. 럭셔리 스타일로 가고 싶어요!', type='TextMessage'), ToolCallRequestEvent(id='e260fff0-23ad-4787-a3a3-eeb4377b9d49', source='budget_planner', models_usage=RequestUsage(prompt_tokens=171, completion_tokens=27), metadata={}, created_at=datetime.datetime(2025, 11, 1, 14, 21, 49, 69573, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_S4x5rGHYGXGHsRVOHgTtmi3o', arguments='{"city":"서울","days":4,"style":"럭셔리"}', name='calculate_travel_budget')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='b52bb2be-8eca-4dcd-9ff4-c2c34ab27272', source='budget_planner', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 14, 21, 49, 71083, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='서울 4일 여행 (럭셔리 스타일

In [ ]:
# 테스트 3: 컨시어지 팀 - 핸드오프 패턴 
print("\n=== 🛎️ 여행 컨시어지 팀 테스트 (핸드오프 패턴) ===")
print("간단한 질문:")
await Console(concierge_team.run_stream(task="여행 가방 짐싸기 팁 알려주세요"))


=== 🛎️ 여행 컨시어지 팀 테스트 (핸드오프 패턴) ===
간단한 질문:
---------- TextMessage (user) ----------
여행 가방 짐싸기 팁 알려주세요
---------- TextMessage (travel_concierge) ----------
여행 가방 짐 싸기에 도움이 될 만한 몇 가지 팁을 드리겠습니다:

1. **패킹 리스트 작성**: 여행 전에 필요한 물품을 미리 리스트로 작성하여 빠뜨리는 것을 방지하세요.

2. **의류 롤팩**: 옷을 롤 형태로 말아Packing하면 공간을 절약할 수 있습니다.

3. **중량 분배**: 무거운 물건은 가방의 바닥에 배치하고, 가벼운 물건은 위쪽에 놓아 균형을 맞추세요.

4. **멀티 기능 아이템**: 한 개의 아이템으로 여러 용도로 사용할 수 있는 다기능 제품을 선택해 짐을 줄이세요. 예를 들어, 스카프는 목도리, 담요, 또는 수건으로도 사용할 수 있습니다.

5. **투명 백 사용**: 액세서리나 작은 물품들은 투명 백에 넣어 정리하면 찾기 쉽고 공간을 효율적으로 쓸 수 있습니다.

6. **신발 공간 활용**: 신발 안쪽에 양말이나 작은 물건을 넣어 공간을 활용하세요.

7. **세탁용 비닐**: 이미 사용한 옷은 세탁용 비닐에 넣어 다른 물품들과 분리하세요.

이 팁들이 도움이 되길 바랍니다! 안전하고 즐거운 여행 되세요!
---------- TextMessage (travel_concierge) ----------
안내 완료


TaskResult(messages=[TextMessage(id='d456b6f2-f335-4293-ac6d-fe2f1efcd54d', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 14, 22, 2, 306369, tzinfo=datetime.timezone.utc), content='여행 가방 짐싸기 팁 알려주세요', type='TextMessage'), TextMessage(id='65fb5654-b685-4029-9b5f-3384189d82aa', source='travel_concierge', models_usage=RequestUsage(prompt_tokens=144, completion_tokens=307), metadata={}, created_at=datetime.datetime(2025, 11, 1, 14, 22, 5, 661015, tzinfo=datetime.timezone.utc), content='여행 가방 짐 싸기에 도움이 될 만한 몇 가지 팁을 드리겠습니다:\n\n1. **패킹 리스트 작성**: 여행 전에 필요한 물품을 미리 리스트로 작성하여 빠뜨리는 것을 방지하세요.\n\n2. **의류 롤팩**: 옷을 롤 형태로 말아Packing하면 공간을 절약할 수 있습니다.\n\n3. **중량 분배**: 무거운 물건은 가방의 바닥에 배치하고, 가벼운 물건은 위쪽에 놓아 균형을 맞추세요.\n\n4. **멀티 기능 아이템**: 한 개의 아이템으로 여러 용도로 사용할 수 있는 다기능 제품을 선택해 짐을 줄이세요. 예를 들어, 스카프는 목도리, 담요, 또는 수건으로도 사용할 수 있습니다.\n\n5. **투명 백 사용**: 액세서리나 작은 물품들은 투명 백에 넣어 정리하면 찾기 쉽고 공간을 효율적으로 쓸 수 있습니다.\n\n6. **신발 공간 활용**: 신발 안쪽에 양말이나 작은 물건을 넣어 공간을 활용하세요.\n\n7. **세탁용 비닐*

In [ ]:
# 복잡한 질문으로 핸드오프 테스트
print("\n복잡한 질문 (핸드오프 예상):")
await Console(concierge_team.run_stream(task="유럽 비자 신청 절차와 항공료 할인받는 방법, 여행자보험 추천까지 상세히 알려주세요"))


복잡한 질문 (핸드오프 예상):
---------- TextMessage (user) ----------
유럽 비자 신청 절차와 항공료 할인받는 방법, 여행자보험 추천까지 상세히 알려주세요
---------- ToolCallRequestEvent (travel_concierge) ----------
[FunctionCall(id='call_J5pEKDJqLOEf68t32cRkZRuR', arguments='{}', name='transfer_to_travel_advisor')]
---------- ToolCallExecutionEvent (travel_concierge) ----------
[FunctionExecutionResult(content='전문 상담사에게 연결합니다', name='transfer_to_travel_advisor', call_id='call_J5pEKDJqLOEf68t32cRkZRuR', is_error=False)]
---------- HandoffMessage (travel_concierge) ----------
전문 상담사에게 연결합니다


TaskResult(messages=[TextMessage(id='76275814-a73d-44c3-9278-866281031b90', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 14, 22, 6, 133090, tzinfo=datetime.timezone.utc), content='유럽 비자 신청 절차와 항공료 할인받는 방법, 여행자보험 추천까지 상세히 알려주세요', type='TextMessage'), ToolCallRequestEvent(id='5fde7c04-f28c-4422-be52-ca62a594f66a', source='travel_concierge', models_usage=RequestUsage(prompt_tokens=490, completion_tokens=15), metadata={}, created_at=datetime.datetime(2025, 11, 1, 14, 22, 7, 47880, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_J5pEKDJqLOEf68t32cRkZRuR', arguments='{}', name='transfer_to_travel_advisor')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='4912e509-435a-4634-a0f3-4cdb17bd9df8', source='travel_concierge', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 14, 22, 7, 49155, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='전문 상담사에게 연결합니다', name='transfer_to_travel_ad

## 🎓 실습 과제 및 확장 아이디어

### 📚 이번 실습에서 배운 내용:
1. **실시간 API 연동**: Google Places API와 OpenWeatherMap API 활용법
2. **Fallback 시스템**: API 키 없이도 동작하는 안전장치 구현
3. **단일 에이전트 팀**: 실제 API를 활용하는 전문 에이전트
4. **반성 팀**: Primary-Critic 협업으로 품질 향상
5. **핸드오프 팀**: 적절한 역할 분담과 에스컬레이션
6. **비동기 처리**: aiohttp를 활용한 효율적인 API 호출

### 🚀 추가 도전 과제:
1. **더 많은 API 연동**: 
   - 실시간 환율 API (ExchangeRate-API)
   - 항공료 검색 API (Amadeus, Skyscanner)
   - 호텔 가격 비교 API (Booking.com, Hotels.com)

2. **고급 에이전트 기능**:
   - 다국어 번역 API 연동
   - 이미지 분석 API (관광지 사진 분석)
   - 추천 시스템 (사용자 취향 기반)

3. **프로덕션 배포**:
   - FastAPI 웹 서버 구축
   - 데이터베이스 연결 (PostgreSQL, MongoDB)
   - Docker 컨테이너화

### 💡 핵심 포인트:
- **명확한 역할 분담**: 각 에이전트의 전문성 정의
- **적절한 종료 조건**: 무한 루프 방지와 효율성
- **사용자 경험 고려**: 언제 전문가에게 연결할지 판단
- **한국어 자연어 처리**: 문화적 맥락을 고려한 대화

---

## 🔥 보너스 실습: 실시간 API 연동 통합 여행 계획 시스템

위에서 만든 **실시간 API 연동 팀들**을 순서대로 실행하여 완전한 여행 계획을 세워보세요!

```python
# 실시간 API 연동 워크플로우:
# 1단계: 실시간 여행지 + 날씨 정보 → 2단계: 예산 계획 → 3단계: 전문 상담

async def complete_travel_planning_with_apis(destination: str, duration: str, style: str):
    print("🌐 1단계: 실시간 여행지 + 날씨 정보")
    # travel_expert_team 실행 (Google Places + OpenWeatherMap API)
    
    print("💰 2단계: 예산 계획") 
    # budget_management_team 실행 (예산 계산 + 검토)
    
    print("🛎️ 3단계: 전문 상담")
    # concierge_team 실행 (복잡한 요청 시 핸드오프)
```

### 🌟 API 키 설정 방법:
```bash
# .env 파일에 추가
GOOGLE_PLACES_API_KEY=your_google_api_key_here
OPENWEATHER_API_KEY=your_openweather_api_key_here
```

**이제 여러분만의 실시간 API 연동 AI 팀을 만들어보세요!** 🚀🌐